In [1]:
# ENV["PYTHON"] = "/home/users/kww231/virtualenvs/phd/bin/python"
# rm(Pkg.dir("PyCall","deps","PYTHON")); Pkg.checkout("PyCall"); Pkg.build("PyCall")

include("../src/mod_funcs.jl")  # this file contains code to access halos and run models
include("../src/obs_funcs.jl")  # this code gets the observed stellar mass function for comparison
include("../src/fit_funcs.jl")  # include functions for fitting models

import BlackBoxOptim

INFO: Recompiling stale cache file /home/users/kww231/.playground/share/phd/packages/lib/v0.6/Rmath.ji for module Rmath.
INFO: Recompiling stale cache file /home/users/kww231/.playground/share/phd/packages/lib/v0.6/Distributions.ji for module Distributions.
INFO: Recompiling stale cache file /home/users/kww231/.playground/share/phd/packages/lib/v0.6/Gadfly.ji for module Gadfly.
INFO: Recompiling stale cache file /home/users/kww231/.playground/share/phd/packages/lib/v0.6/BlackBoxOptim.ji for module BlackBoxOptim.

Use "DictChain{K,V}(...) where {K,V}" instead.

Use "DictChain{K,V}(...) where {K,V}" instead.

Use "DictChain{K,V}(...) where {K,V}" instead.

Use "Candidate{F}(...) where F" instead.

Use "FrontierIndividual{F}(...) where F" instead.

Use "SimplexCrossover{NP}(...) where NP" instead.

Use "SimplexCrossover{NP}(...) where NP" instead.

Use "DiffEvoRandBin{N}(...) where N" instead.

Use "DiffEvoRandBin{N}(...) where N" instead.

Use "ParentCentricCrossover{NP}(...) where NP" i

In [2]:
# load halos
halodf = load_halodat()

,halo_id,halo_upid,x,y,z,vx,vy,vz,hfoo1,hfoo2,halo_mvir,halo_mz_05,halo_mz_1,halo_mz_2,halo_mz_4,halo_mz_8,halo_ac,halo_ac_mpeak,halo_nfw_conc,halo_spin,hfoo3,halo_rvir,logMh,Mpeak,logMpeak,Mbar
1,1620490488,-1,23.8908,178.352,71.8972,254.19,-195.17,-253.54,758.46,780.72,1.303e14,1.117e14,1.016e14,9.289e13,6.113e13,4.885e13,0.51462,0.51462,4.34886,0.03889,1620490488,1.25141,14.1149,1.303e14,14.1149,2.03268e13
2,1620489517,-1,54.2385,147.212,83.8446,132.18,-243.42,241.66,746.88,787.27,1.031e14,1.009e14,9.834e13,8.531e13,6.773e13,5.168e13,0.602988,0.602988,6.31183,0.03163,1620489517,1.15745,14.0133,1.031e14,14.0133,1.60836e13
3,1620568787,-1,17.5035,142.147,121.233,206.25,52.6,75.76,800.83,833.51,1.023e14,9.305e13,8.565e13,8.209e13,7.753e13,5.354e13,0.465599,0.465599,8.92032,0.01292,1620568787,1.15445,14.0099,1.023e14,14.0099,1.59588e13
4,1620569234,-1,30.9299,185.085,116.149,100.4,211.18,314.45,759.85,808.61,8.734e13,8.65e13,8.481e13,7.882e13,6.102e13,4.87e13,0.474902,0.474902,8.7816,0.02291,1620569234,1.09519,13.9412,8.734e13,13.9412,1.3625e13
5,1620569405,-1,3.68962,147.032,119.642,318.78,-36.68,214.57,655.71,762.05,7.256e13,7.268e13,7.435e13,8.4e13,7.07e13,2.074e13,0.579871,0.608062,5.65232,0.0223,1620569405,1.02956,13.8607,8.4e13,13.9243,1.3104e13
6,1619882298,-1,46.9333,134.869,85.2907,-32.87,0.42,29.38,664.42,737.05,7.239e13,6.482e13,6.325e13,6.602e13,6.553e13,2.278e13,0.734503,0.734503,6.13975,0.06525,1619882298,1.02875,13.8597,7.239e13,13.8597,1.12928e13
7,1620570495,-1,10.0025,186.166,125.959,173.74,249.63,270.22,650.17,679.69,6.975e13,6.799e13,6.33e13,6.143e13,5.133e13,3.735e13,0.529195,0.529195,6.26543,0.04754,1620570495,1.01609,13.8435,6.975e13,13.8435,1.0881e13
8,1621113337,-1,39.2183,196.085,87.0058,-92.91,99.81,237.01,560.58,643.33,5.08e13,5.55e13,5.615e13,6.024e13,3.472e13,1.32e13,0.503591,0.516679,4.35401,0.08217,1621113337,0.914195,13.7059,6.024e13,13.7799,9.39744e12
9,1620494893,-1,38.9772,181.184,67.201,-119.43,12.79,11.14,568.63,639.93,4.591e13,4.567e13,4.37e13,4.085e13,3.21e13,1.713e13,0.473207,0.475291,6.38986,0.03874,1620494893,0.883866,13.6619,4.591e13,13.6619,7.16196e12
10,1620494673,-1,28.9453,165.548,73.521,28.3,-4.23,-105.61,586.93,628.04,4.428e13,4.125e13,3.994e13,3.841e13,3.449e13,2.469e13,0.381623,0.381623,7.59335,0.03238,1620494673,0.87328,13.6462,4.428e13,13.6462,6.90768e12


In [3]:
# set the data sample for which to optimise the mass parameters
sample_num = 3

3

In [4]:
# set the range of galaxy stellar masses for the model, based on the sample being compared with
min_masses = [9.4, 9.8, 10.3]
const mockmassbins = [Array(min_masses[sample_num]:0.1:11.0)]

1-element Array{Array{Float64,1},1}:
 [10.3, 10.4, 10.5, 10.6, 10.7, 10.8, 10.9, 11.0]

In [5]:
# define arrays of the peak halo mass and baryonic mass that can be passed to model functions
const mockmpeaks = [Array(halodf[:Mpeak])]
const mockmbars = [Array(halodf[:Mbar])]

1-element Array{Array{Float64,1},1}:
 [2.03268e13, 1.60836e13, 1.59588e13, 1.3625e13, 1.3104e13, 1.12928e13, 1.0881e13, 9.39744e12, 7.16196e12, 6.90768e12  …  1.77996e10, 1.63332e10, 1.57716e10, 1.58652e10, 1.62396e10, 1.89072e10, 1.65204e10, 1.5678e10, 1.57716e10, 1.5678e10]

In [6]:
# define new method of massbincounts function for MCMC sampling using the above defined arrays
massbincounts(params) = massbincounts(mockmpeaks, mockmbars,
                                      mockmassbins,
                                      params, fix_sig = false)

massbincounts (generic function with 2 methods)

In [7]:
# take the data sample to compare with
datdf, samplevols = load_groupdat(sample_num, quenform = "both")  # here integer 2 means to load only the second sample
                                                                  # (magnitude threshold -19) and "both" means to load
                                                                  # all galaxies, both quenched and starforming

(47784×46 DataFrames.DataFrame. Omitted printing of 39 columns
│ Row   │ foo    │ galid  │ groupid │ cenid  │ rband    │ Psat │ Mh         │
├───────┼────────┼────────┼─────────┼────────┼──────────┼──────┼────────────┤
│ 1     │ PROB10 │ 3      │ 26747   │ 3      │ -20.9869 │ 0.0  │ 3.03811e12 │
│ 2     │ PROB10 │ 17     │ 46451   │ 17     │ -20.4593 │ 0.0  │ 1.67686e12 │
│ 3     │ PROB10 │ 21     │ 29108   │ 21     │ -21.3315 │ 0.0  │ 2.77697e12 │
│ 4     │ PROB10 │ 53     │ 15187   │ 53     │ -21.1077 │ 0.0  │ 5.48889e12 │
│ 5     │ PROB10 │ 81     │ 36849   │ 81     │ -20.5768 │ 0.0  │ 2.15532e12 │
│ 6     │ PROB10 │ 83     │ 24536   │ 83     │ -20.8211 │ 0.0  │ 3.32973e12 │
│ 7     │ PROB10 │ 85     │ 52256   │ 85     │ -20.6159 │ 0.0  │ 1.47419e12 │
│ 8     │ PROB10 │ 90     │ 25369   │ 90     │ -20.8662 │ 0.0  │ 3.21402e12 │
│ 9     │ PROB10 │ 93     │ 10451   │ 93     │ -21.3781 │ 0.0  │ 7.98381e12 │
│ 10    │ PROB10 │ 112    │ 1095    │ 112    │ -21.8533 │ 0.0  │ 4.79077e13 │
│

In [8]:
# compute the counts of galaxies within each massbin and the volume factor of that sample's volume to the simulation's
const obsdat = histbincounts(datdf[:log10M], mockmassbins[1])
const volfac = samplevols[sample_num] / 400^3

# compute variables for the cstat function, first a mask of nonzero elements (redundant in this case)
const nonzmask = find(obsdat .> 0)

# and a tuple of the sums of the data and the log of the data
const obsums = (sum(obsdat), sum(obsdat[nonzmask] .* (log.(obsdat[nonzmask]))))

(47784, 433184.7166672442)

In [9]:
# define a likelihood function for the stellar mass model
function like(params::Array{Float64, 1})
    
    gendat = massbincounts(params)
    if any(gendat .== 0)  # if there are bins with no galaxies, something is wrong
        return 1e30
    end

    gendat = volfac .* gendat  # convert galaxy counts to expected rates in sample volume

    score = cstat(gendat, obsdat, obsums, nonzmask)  # compute cstat of model and data
    score
end

like (generic function with 1 method)

In [35]:
# take the best fit parameters of the Moster et al. paper to initialise the optimiser with
initmeans = [11.8, 0.14, 1.75, 0.57, 10.8, 0.16, 1.0]

7-element Array{Float64,1}:
 11.8 
  0.14
  1.75
  0.57
 10.8 
  0.16
  1.0 

In [86]:
# set up the optimiser
ndims = 7
npop = 100

x0 = randn(ndims, npop)
x0[1, :] = 0.01351 * x0[1, :] + initmeans[1]  # M₁
x0[2, :] = 0.01351 * x0[2, :] + initmeans[2]  # ϵ
x0[3, :] = 0.01351 * x0[3, :] + initmeans[3]  # β
x0[4, :] = 0.01351 * x0[4, :] + initmeans[4]  # γ
x0[5, :] = 0.01351 * x0[5, :] + initmeans[5]  # Mσ
x0[6, :] = 0.01351 * x0[6, :] + initmeans[6]  # σ₀
x0[7, :] = 0.01351 * x0[7, :] + initmeans[7]  # α
x0[x0 .<= 0] = 1e-5  # lose any non-positive values

# set the boundaries of the search space for optimisation
lower = 1e-5 * ones(ndims)
upper = 1.0 * [20, 2, 5, 3, 20, 2, 3]

optctrl = BlackBoxOptim.bbsetup(like;  # Method = :dxnes,
                                SearchRange = collect(zip(lower, upper)),
                                PopulationSize = npop, Population = x0)

BlackBoxOptim.OptController{BlackBoxOptim.DiffEvoOpt{BlackBoxOptim.FitPopulation{Float64},BlackBoxOptim.RadiusLimitedSelector,BlackBoxOptim.AdaptiveDiffEvoRandBin{3},BlackBoxOptim.RandomBound{BlackBoxOptim.RangePerDimSearchSpace}},BlackBoxOptim.FunctionBasedProblem{BlackBoxOptim.ScalarFitnessScheme{true},BlackBoxOptim.RangePerDimSearchSpace,Void}}(BlackBoxOptim.DiffEvoOpt{BlackBoxOptim.FitPopulation{Float64},BlackBoxOptim.RadiusLimitedSelector,BlackBoxOptim.AdaptiveDiffEvoRandBin{3},BlackBoxOptim.RandomBound{BlackBoxOptim.RangePerDimSearchSpace}}("AdaptiveDE/rand/1/bin/radiuslimited", BlackBoxOptim.FitPopulation{Float64}([12.0591 12.0523 … 12.0227 12.0256; 0.210229 0.166607 … 0.187556 0.186441; … ; 0.321611 0.328277 … 0.307352 0.306816; 1.62274 1.6502 … 1.61333 1.6228], NaN, [NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN  …  NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN], 0, BlackBoxOptim.Candidate{Float64}[]), BlackBoxOptim.RadiusLimitedSelector(8), BlackBoxOptim.AdaptiveDiffEvo

In [87]:
# run optimiser for some number of steps
res = BlackBoxOptim.bboptimize(optctrl; MaxSteps = 400)

Starting optimization with optimizer BlackBoxOptim.DiffEvoOpt{BlackBoxOptim.FitPopulation{Float64},BlackBoxOptim.RadiusLimitedSelector,BlackBoxOptim.AdaptiveDiffEvoRandBin{3},BlackBoxOptim.RandomBound{BlackBoxOptim.RangePerDimSearchSpace}}
0.00 secs, 0 evals, 0 steps
28.71 secs, 2 evals, 1 steps, improv/step: 1.000 (last = 1.0000), fitness=107.307651165
46.32 secs, 4 evals, 2 steps, improv/step: 0.500 (last = 0.0000), fitness=107.307651165
52.22 secs, 6 evals, 3 steps, improv/step: 0.333 (last = 0.0000), fitness=107.307651165
58.14 secs, 8 evals, 4 steps, improv/step: 0.250 (last = 0.0000), fitness=107.307651165
64.15 secs, 10 evals, 5 steps, improv/step: 0.200 (last = 0.0000), fitness=82.205051503
70.04 secs, 12 evals, 6 steps, improv/step: 0.167 (last = 0.0000), fitness=82.205051503
75.93 secs, 14 evals, 7 steps, improv/step: 0.286 (last = 1.0000), fitness=74.805516520
82.07 secs, 16 evals, 8 steps, improv/step: 0.250 (last = 0.0000), fitness=44.559335705
88.05 secs, 18 evals, 9 step

522.67 secs, 165 evals, 89 steps, improv/step: 0.337 (last = 1.0000), fitness=20.085485351
528.50 secs, 167 evals, 90 steps, improv/step: 0.333 (last = 0.0000), fitness=20.085485351
534.35 secs, 169 evals, 91 steps, improv/step: 0.330 (last = 0.0000), fitness=20.085485351
540.17 secs, 171 evals, 92 steps, improv/step: 0.326 (last = 0.0000), fitness=20.085485351
545.16 secs, 172 evals, 93 steps, improv/step: 0.323 (last = 0.0000), fitness=20.085485351
551.06 secs, 174 evals, 94 steps, improv/step: 0.319 (last = 0.0000), fitness=20.085485351
556.89 secs, 176 evals, 95 steps, improv/step: 0.316 (last = 0.0000), fitness=20.085485351
562.70 secs, 178 evals, 96 steps, improv/step: 0.313 (last = 0.0000), fitness=20.085485351
568.51 secs, 180 evals, 97 steps, improv/step: 0.309 (last = 0.0000), fitness=20.085485351
574.42 secs, 182 evals, 98 steps, improv/step: 0.306 (last = 0.0000), fitness=20.085485351
580.35 secs, 184 evals, 99 steps, improv/step: 0.313 (last = 1.0000), fitness=20.085485351

1369.23 secs, 446 evals, 267 steps, improv/step: 0.352 (last = 1.0000), fitness=18.942189770
1370.25 secs, 447 evals, 268 steps, improv/step: 0.351 (last = 0.0000), fitness=18.942189770
1376.23 secs, 449 evals, 269 steps, improv/step: 0.349 (last = 0.0000), fitness=18.942189770
1382.20 secs, 451 evals, 270 steps, improv/step: 0.348 (last = 0.0000), fitness=18.942189770
1387.22 secs, 452 evals, 271 steps, improv/step: 0.347 (last = 0.0000), fitness=18.942189770
1388.16 secs, 453 evals, 272 steps, improv/step: 0.346 (last = 0.0000), fitness=18.942189770
1394.17 secs, 455 evals, 273 steps, improv/step: 0.348 (last = 1.0000), fitness=18.942189770
1400.40 secs, 456 evals, 274 steps, improv/step: 0.347 (last = 0.0000), fitness=18.942189770
1401.43 secs, 457 evals, 275 steps, improv/step: 0.345 (last = 0.0000), fitness=18.942189770
1406.89 secs, 458 evals, 276 steps, improv/step: 0.344 (last = 0.0000), fitness=18.942189770
1414.58 secs, 460 evals, 277 steps, improv/step: 0.347 (last = 1.0000)

1929.42 secs, 563 evals, 356 steps, improv/step: 0.334 (last = 0.0000), fitness=16.820240950
1953.89 secs, 565 evals, 357 steps, improv/step: 0.333 (last = 0.0000), fitness=16.820240950
1957.06 secs, 566 evals, 358 steps, improv/step: 0.335 (last = 1.0000), fitness=16.820240950
1961.80 secs, 567 evals, 359 steps, improv/step: 0.337 (last = 1.0000), fitness=16.820240950
2029.28 secs, 568 evals, 360 steps, improv/step: 0.336 (last = 0.0000), fitness=16.820240950
2037.22 secs, 570 evals, 361 steps, improv/step: 0.338 (last = 1.0000), fitness=16.820240950
2043.74 secs, 572 evals, 362 steps, improv/step: 0.337 (last = 0.0000), fitness=16.820240950
2049.05 secs, 573 evals, 363 steps, improv/step: 0.339 (last = 1.0000), fitness=16.820240950
2050.09 secs, 574 evals, 364 steps, improv/step: 0.338 (last = 0.0000), fitness=16.820240950
2055.16 secs, 575 evals, 365 steps, improv/step: 0.337 (last = 0.0000), fitness=16.820240950
2061.29 secs, 577 evals, 366 steps, improv/step: 0.339 (last = 1.0000)

BlackBoxOptim.OptimizationResults("adaptive_de_rand_1_bin_radiuslimited", "Max number of steps (400) reached", 401, 1.533659277124342e9, 2201.1656079292297, BlackBoxOptim.DictChain{Symbol,Any}[Dict{Symbol,Any}(Pair{Symbol,Any}(:RngSeed, 770784),Pair{Symbol,Any}(:MaxSteps, 400)),BlackBoxOptim.DictChain{Symbol,Any}[BlackBoxOptim.DictChain{Symbol,Any}[Dict{Symbol,Any}(Pair{Symbol,Any}(:SearchRange, Tuple{Float64,Float64}[(1.0e-5, 20.0), (1.0e-5, 2.0), (1.0e-5, 5.0), (1.0e-5, 3.0), (1.0e-5, 20.0), (1.0e-5, 2.0), (1.0e-5, 3.0)]),Pair{Symbol,Any}(:Population, [12.0591 12.0523 … 12.0227 12.0087; 0.210229 0.180631 … 0.187556 0.183879; … ; 0.321611 0.328277 … 0.307352 0.324443; 1.62335 1.6502 … 1.61333 1.67592]),Pair{Symbol,Any}(:MaxSteps, 10000),Pair{Symbol,Any}(:PopulationSize, 100)),Dict{Symbol,Any}()],Dict{Symbol,Any}(Pair{Symbol,Any}(:FitnessScheme, BlackBoxOptim.ScalarFitnessScheme{true}()),Pair{Symbol,Any}(:NumDimensions, :NotSpecified),Pair{Symbol,Any}(:PopulationSize, 50),Pair{Symbol,A


Optimization stopped after 401 steps and 2201.1656079292297 seconds
Termination reason: Max number of steps (400) reached
Steps per second = 0.1821762063497099
Function evals per second = 0.28212325222735624
Improvements/step = 0.345
Total function evaluations = 621


Best candidate found: [12.04, 0.184506, 1.25783, 1.37414, 10.0241, 0.321572, 1.6363]

Fitness: 16.820240950



In [57]:
# grab the best candidate to use as initial position for next run
initmeans = BlackBoxOptim.best_candidate(res)

7-element Array{Float64,1}:
 12.0357  
  0.184856
  1.25241 
  1.38012 
 10.0068  
  0.323894
  1.63352 

In [21]:
# save best result
writedlm("/home/kilian/wrk/assbiass/dat/optim/M20_mass_pars.dat", initmeans)